In [2]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import metrics
from sklearn.model_selection import train_test_split
import catboost as cb
import os
from IPython import embed
from datetime import date
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import statsmodels
import warnings
warnings.filterwarnings('ignore')

In [3]:
base_path = os.getcwd()
selected_columns = ['precipitacao (mm)-1','temperatura (°C)-1','umidade ar (%)-1','t-1','t-2','t-3','densidade_demografica']
target = 'dengue_diagnosis'
df_input = pd.read_csv(base_path+'/dengue_RIO_POA.csv',sep=";")
months = list(range(1, 13))

In [4]:
def prepare_df(ano_teste=2020,mes_inicio_teste=1, meses_previsao=1):
    df_input['data'] = df_input[['ano', 'mes']].apply(lambda x: date(x[0], x[1], 1), axis=1)
    list_data_teste = list()
    for m in list(range(mes_inicio_teste, mes_inicio_teste+3)):
            if m < 13:
                list_data_teste.append(date(ano_teste, m, 1))
            else:
                list_data_teste.append(date(ano_teste+1, m-12, 1))
    
    print(list_data_teste)
    X_train = df_input[~df_input['data'].isin(list_data_teste)]
    Y_train = df_input[~df_input['data'].isin(list_data_teste)]
    
   
    X_train = X_train[selected_columns]
    Y_train = Y_train[target]
    print("Len X_TRAIN", len(X_train))
    print("X_TRAIN: ", X_train)
    print("Y_TRAIN: ", Y_train)

    return X_train, Y_train

In [5]:
def base_prepare(df_input, ano_teste=2021, mes_inicio_teste=1, meses_previsao=1):
    df_input['data'] = df_input[['ano', 'mes']].apply(lambda x: date(x[0], x[1], 1), axis=1)
    
    list_data_teste = list()
    for m in list(range(mes_inicio_teste, mes_inicio_teste+meses_previsao)):
            if m < 13:
                list_data_teste.append(date(ano_teste, m, 1))
            else:
                list_data_teste.append(date(ano_teste+1, m-12, 1))
                
    # Filter dates which is in test data list
    df_teste = df_input[df_input['data'].isin(list_data_teste)]

    # Filter dates which is not in test data list
    df_treino = df_input[~df_input['data'].isin(list_data_teste)]
    print('df_treino',df_treino)
    print('df_teste',df_teste)
    print("LEN: ", len(df_treino) , len(df_teste), len(df_input))
    
    df_treino.drop(columns=['data'], inplace=True)
    df_teste.drop(columns=['data'], inplace=True)

    return df_treino, df_teste

In [6]:
def main_step(ano=2020,mes_inicio_teste=1):
    X_train, Y_train = prepare_df(ano_teste=ano,mes_inicio_teste=mes_inicio_teste)
    
    
    df_treino, df_teste = base_prepare(df_input,ano_teste=ano,mes_inicio_teste=mes_inicio_teste)
    
    #x_train_sample, x_test_sample, y_train_sample, y_test_sample = train_test_split(X_train,
    #                                                                                    Y_train,
    #                                                                                    test_size=0.15,
    #                                                                                    random_state=42)
    x_train_sample = X_train.copy()
    y_train_sample = Y_train.copy()
    return df_treino, df_teste, x_train_sample, y_train_sample

In [7]:
def main_step_2(df_test,x_train,y_train):
    regressor, r2 = run_catboost(x_train, y_train, grid_search=False, standart=False)
    df_previsao_X, columns = prepare_base_test(df_test)
    df_previsao_X['date'] = df_previsao_X[['ano', 'mes']].apply(lambda x: date(x[0], x[1], 1), axis=1)
    list_results = list()
    df_result_consolidate = pd.DataFrame(
        columns=list(df_previsao_X.columns)+[target+"_previsto"])
    
    df_result_consolidate.drop(columns=['date'], inplace=True)
    
    
    for d in list(df_previsao_X['date'].unique()):
        #print(d)
        df_base = df_previsao_X[df_previsao_X['date'] == d]
        
        #df = df_previsao_X[df_previsao_X['date'] == list(df_previsao_X['date'].unique())[0]]
        df = df_previsao_X[df_previsao_X['date'] == d]
        df.drop(columns=['date'], inplace=True)
        

        #if len(list_results) > 0:

            # Atribui os valores que já foram previstos para o novo df
            #for n in list(range(1, len(list_results)+1)):
            #    df['t-{}'.format(n)] = list_results[-n]
        
        df_prev = df.drop(columns=['nome_distrito', 'chave', 'ano', 'mes', 'cod_distrito'])
        df_prev = df_prev[columns]
        # Faz a previsão e armazena na lista de suporte e no df de resultado final
        list_results.append(regressor.predict(df_prev))
        if target == 'dengue_diagnosis':
            list_results[-1] = [0 if a < 0 else round(a) for a in list_results[-1]]
        df_base[target+"_previsto"] = list_results[-1]
        #print(df_prev)
        #print(list_results[-1])
        df_result_consolidate = df_result_consolidate.append(df_base)
        
    df_result_consolidate.sort_values(by=["cod_distrito", "ano", "mes"], inplace=True)
    df_test = pd.merge(df_test, df_result_consolidate[["chave", target+'_previsto']], on="chave", how="left")

    print(df_test)
    return df_test

In [8]:
def build_model_prediction():
    first = True
    for i in range(2017,2023+1):
        for initial_test_month in months:
            df_train, df_test, x_train, y_train = main_step(i,mes_inicio_teste=initial_test_month)

            df_result = main_step_2(df_test, x_train, y_train)

            variables = ['nome_distrito', 'ano', 'mes_inicial', 'mes', target, target + '_previsto']
            variables.extend(selected_columns)
            df_consolidate = pd.DataFrame(columns=variables)

            df_result['mes_inicial'] = [1] * df_result.shape[0]

            df_consolidate = df_consolidate.append(df_result[variables])
            if first:
                df_consolidate.to_csv(base_path + f"/data/result_prediction_simple.csv",mode='w',header=True)
            else:
                df_consolidate.to_csv(base_path + f"/data/result_prediction_simple.csv",mode='a',header=False)
            first = False


In [9]:
def prepare_base_test(df_previsao):
    df_previsao_X = df_previsao[selected_columns]
    mes_list = ["mes_{}".format(i) for i in list(df_previsao["mes"].unique())]

    columns, index_time_cols = get_indices_columns_time(df_previsao_X.columns)
    df_previsao_X = df_previsao_X[columns]

    
    # Precisa dessas variáveis para percorrer na previsão
    df_previsao_X['ano'] = [0] * df_previsao_X.shape[0]
    df_previsao_X['mes'] = [0] * df_previsao_X.shape[0]
    df_previsao_X['nome_distrito'] = [0] * df_previsao_X.shape[0]
    df_previsao_X['chave'] = [0] * df_previsao_X.shape[0]
    df_previsao_X['cod_distrito'] = [0] * df_previsao_X.shape[0]

    # Precisa dessas variáveis para percorrer na previsão
    df_previsao_X.loc[:,['ano']] = list(df_previsao['ano'])
    df_previsao_X.loc[:,['mes']] = list(df_previsao['mes'])
    df_previsao_X.loc[:,['nome_distrito']] = list(df_previsao['nome_distrito'])
    df_previsao_X.loc[:,['chave']] = list(df_previsao['chave'])
    df_previsao_X.loc[:,['cod_distrito']] = list(df_previsao['cod_distrito'])
    
    return df_previsao_X, columns

In [10]:
def get_indices_columns_time(columns):
    columns_time = list()
    columns_others = list()

    for c in columns:
        if re.match("t-.", c):
            columns_time.append(c)
        else:
            columns_others.append(c)

    if len(columns_time) > 0:

        index_time_cols = (len(columns_others),
                           len(columns_others) + len(columns_time) - 1)

        columns_others = columns_others + columns_time

        return columns_others, index_time_cols

    else:
        return columns_others, list()

In [11]:
def run_catboost(x_train, y_train, grid_search=False, standart=False):
    
    x_train.to_csv(base_path + f"/x_train.csv")
    y_train.to_csv(base_path + f"/y_train.csv")
    if standart:
        regressor = cb.CatBoostRegressor()

    else:
        regressor = cb.CatBoostRegressor(learning_rate=0.1,
                                         bootstrap_type='Bernoulli',
                                         grow_policy='Lossguide',
                                         boosting_type='Plain',
                                         )

    regressor.fit(x_train, y_train,plot=True)

    return regressor,0


In [12]:
build_model_prediction()


[datetime.date(2017, 1, 1), datetime.date(2017, 2, 1), datetime.date(2017, 3, 1)]
Len X_TRAIN 17789
X_TRAIN:         precipitacao (mm)-1  temperatura (°C)-1  umidade ar (%)-1   t-1   t-2  \
0                     25.0              242.00             70.00   5.0   2.0   
1                     26.0              216.00             72.00   9.0   5.0   
2                     28.0               26.00             61.00  47.0   9.0   
3                     26.0              161.00             69.00  69.0  47.0   
4                     25.0               24.00             74.00  96.0  69.0   
...                    ...                 ...               ...   ...   ...   
18315                 89.8               25.47             75.70  13.0   4.0   
18316                 35.2               20.60             73.22  29.0  13.0   
18317                106.8               18.39             80.68  25.0  29.0   
18318                247.6               16.29             80.11   6.0  25.0   
18319     

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 75.5138223	total: 49.5ms	remaining: 49.5s
1:	learn: 71.7579669	total: 52.4ms	remaining: 26.2s
2:	learn: 68.1269697	total: 55.2ms	remaining: 18.3s
3:	learn: 64.6809241	total: 57.1ms	remaining: 14.2s
4:	learn: 61.9829290	total: 59.1ms	remaining: 11.8s
5:	learn: 59.0468091	total: 61.1ms	remaining: 10.1s
6:	learn: 56.3424093	total: 63.3ms	remaining: 8.97s
7:	learn: 54.2720537	total: 65.3ms	remaining: 8.1s
8:	learn: 52.3575368	total: 67.3ms	remaining: 7.41s
9:	learn: 50.1722657	total: 69.3ms	remaining: 6.86s
10:	learn: 47.7716972	total: 71.2ms	remaining: 6.4s
11:	learn: 45.6446458	total: 73.4ms	remaining: 6.04s
12:	learn: 44.2689623	total: 75.4ms	remaining: 5.72s
13:	learn: 43.0856786	total: 77.4ms	remaining: 5.45s
14:	learn: 41.8114503	total: 79ms	remaining: 5.19s
15:	learn: 40.6902310	total: 81.1ms	remaining: 4.99s
16:	learn: 39.5831371	total: 83.1ms	remaining: 4.81s
17:	learn: 38.5150848	total: 85.3ms	remaining: 4.65s
18:	learn: 37.5850116	total: 87.2ms	remaining: 4.5s
19:	lear

AttributeError: 'DataFrame' object has no attribute 'append'

0:	learn: 74.7462194	total: 20.3ms	remaining: 20.3s
1:	learn: 70.6666221	total: 47.3ms	remaining: 23.6s
2:	learn: 66.9636999	total: 64.4ms	remaining: 21.4s
3:	learn: 64.1751987	total: 79.3ms	remaining: 19.8s
4:	learn: 61.1460222	total: 95.2ms	remaining: 18.9s
5:	learn: 58.2523378	total: 116ms	remaining: 19.2s
6:	learn: 56.0997833	total: 132ms	remaining: 18.7s
7:	learn: 53.3317177	total: 147ms	remaining: 18.2s
8:	learn: 51.2969603	total: 175ms	remaining: 19.2s
9:	learn: 49.3489325	total: 190ms	remaining: 18.8s
10:	learn: 47.6725879	total: 206ms	remaining: 18.5s
11:	learn: 45.7541549	total: 220ms	remaining: 18.1s
12:	learn: 44.2693016	total: 251ms	remaining: 19s
13:	learn: 42.9982121	total: 266ms	remaining: 18.7s
14:	learn: 41.4893379	total: 277ms	remaining: 18.2s
15:	learn: 40.2780110	total: 289ms	remaining: 17.8s
16:	learn: 39.2060625	total: 302ms	remaining: 17.5s
17:	learn: 38.0294966	total: 316ms	remaining: 17.2s
18:	learn: 37.1635430	total: 328ms	remaining: 17s
19:	learn: 36.0169475

...

497:	learn: 8.5639767	total: 12.4s	remaining: 12.5s
498:	learn: 8.5557208	total: 12.5s	remaining: 12.5s
499:	learn: 8.5512525	total: 12.5s	remaining: 12.5s
500:	learn: 8.5486424	total: 12.5s	remaining: 12.5s
501:	learn: 8.5412418	total: 12.6s	remaining: 12.5s
502:	learn: 8.5303102	total: 12.6s	remaining: 12.4s
503:	learn: 8.5107641	total: 12.6s	remaining: 12.4s
504:	learn: 8.5081633	total: 12.6s	remaining: 12.4s
505:	learn: 8.4916536	total: 12.6s	remaining: 12.3s
506:	learn: 8.4856974	total: 12.6s	remaining: 12.3s
507:	learn: 8.4692083	total: 12.8s	remaining: 12.4s
508:	learn: 8.4608467	total: 12.8s	remaining: 12.4s
509:	learn: 8.4406338	total: 12.8s	remaining: 12.3s
510:	learn: 8.4317049	total: 12.9s	remaining: 12.3s
511:	learn: 8.4116401	total: 12.9s	remaining: 12.3s
512:	learn: 8.3917079	total: 12.9s	remaining: 12.3s
513:	learn: 8.3848817	total: 13s	remaining: 12.3s
514:	learn: 8.3715128	total: 13.1s	remaining: 12.3s
515:	learn: 8.3631017	total: 13.1s	remaining: 12.3s
516:	learn: 8.

664:	learn: 7.1072241	total: 18.3s	remaining: 9.21s
665:	learn: 7.1028225	total: 18.3s	remaining: 9.17s
666:	learn: 7.0951104	total: 18.3s	remaining: 9.14s
667:	learn: 7.0839622	total: 18.3s	remaining: 9.11s
668:	learn: 7.0788591	total: 18.3s	remaining: 9.07s
669:	learn: 7.0721198	total: 18.4s	remaining: 9.04s
670:	learn: 7.0686121	total: 18.4s	remaining: 9s
671:	learn: 7.0630459	total: 18.4s	remaining: 8.97s
672:	learn: 7.0512223	total: 18.4s	remaining: 8.93s
673:	learn: 7.0492613	total: 18.4s	remaining: 8.9s
674:	learn: 7.0368706	total: 18.4s	remaining: 8.86s
675:	learn: 7.0313577	total: 18.4s	remaining: 8.83s
676:	learn: 7.0239451	total: 18.4s	remaining: 8.79s
677:	learn: 7.0171469	total: 18.4s	remaining: 8.76s
678:	learn: 7.0053602	total: 18.5s	remaining: 8.72s
679:	learn: 7.0032384	total: 18.5s	remaining: 8.69s
680:	learn: 6.9907430	total: 18.5s	remaining: 8.65s
681:	learn: 6.9887629	total: 18.5s	remaining: 8.62s
682:	learn: 6.9861624	total: 18.5s	remaining: 8.59s
683:	learn: 6.97

827:	learn: 6.1022925	total: 20.9s	remaining: 4.35s
828:	learn: 6.0949827	total: 21s	remaining: 4.32s
829:	learn: 6.0911825	total: 21s	remaining: 4.3s
830:	learn: 6.0893838	total: 21s	remaining: 4.27s
831:	learn: 6.0848002	total: 21s	remaining: 4.24s
832:	learn: 6.0814021	total: 21s	remaining: 4.21s
833:	learn: 6.0724825	total: 21s	remaining: 4.19s
834:	learn: 6.0695540	total: 21.1s	remaining: 4.16s
835:	learn: 6.0649083	total: 21.1s	remaining: 4.13s
836:	learn: 6.0612974	total: 21.1s	remaining: 4.1s
837:	learn: 6.0572900	total: 21.1s	remaining: 4.08s
838:	learn: 6.0548542	total: 21.1s	remaining: 4.05s
839:	learn: 6.0525243	total: 21.1s	remaining: 4.02s
840:	learn: 6.0481964	total: 21.1s	remaining: 3.99s
841:	learn: 6.0408862	total: 21.1s	remaining: 3.97s
842:	learn: 6.0358359	total: 21.2s	remaining: 3.94s
843:	learn: 6.0305314	total: 21.2s	remaining: 3.92s
844:	learn: 6.0254404	total: 21.2s	remaining: 3.89s
845:	learn: 6.0215827	total: 21.2s	remaining: 3.86s
846:	learn: 6.0188724	tota

987:	learn: 5.3884785	total: 25s	remaining: 303ms
988:	learn: 5.3839769	total: 25s	remaining: 278ms
989:	learn: 5.3783293	total: 25s	remaining: 253ms
990:	learn: 5.3739236	total: 25s	remaining: 227ms
991:	learn: 5.3717013	total: 25.1s	remaining: 202ms
992:	learn: 5.3703410	total: 25.2s	remaining: 177ms
993:	learn: 5.3628623	total: 25.4s	remaining: 153ms
994:	learn: 5.3574550	total: 25.5s	remaining: 128ms
995:	learn: 5.3519527	total: 25.5s	remaining: 102ms
996:	learn: 5.3477959	total: 25.5s	remaining: 76.8ms
997:	learn: 5.3445081	total: 25.6s	remaining: 51.3ms
998:	learn: 5.3390506	total: 25.7s	remaining: 25.8ms
999:	learn: 5.3372321	total: 25.8s	remaining: 0us


ValueError: Expected a 1D array, got an array with shape (0, 2)

In [12]:
def test_overfit():
    for initial_test_month in months:
        first = True
        df_train, df_test, x_train, y_train = main_step(2022,initial_test_month)

        print("df_train",df_train)
        df_test = df_test.drop(columns=['dengue_diagnosis'])
        print("df_test",df_test)
        print("x_train",x_train)
        print("y_train",y_train)
        df_result = main_step_2(df_test, x_train, y_train)

        variables = ['nome_distrito', 'ano', 'mes_inicial', 'mes',  target + '_previsto']
        variables.extend(selected_columns)
        df_consolidate = pd.DataFrame(columns=variables)

        df_result['mes_inicial'] = [1] * df_result.shape[0]

        df_consolidate = df_consolidate.append(df_result[variables])
        if first:
            df_consolidate.to_csv(base_path + f"/data/prev2023_nextMonth.csv",mode='w',header=True)
        else:
            df_consolidate.to_csv(base_path + f"/data/prev2023_nextMonth.csv",mode='a',header=False)
        first = False


In [13]:
test_overfit()

X_TRAIN:        precipitacao (mm)-1  temperatura (°C)-1  umidade ar (%)-1  t-1  t-2  \
0                   194.8               25.37             77.69   22    4   
1                   194.8               25.37             77.69   22   22   
2                    43.0               26.21             77.29   17   22   
3                   162.2               23.32             78.28    8   17   
4                   113.0               20.35             78.93    4    8   
...                   ...                 ...               ...  ...  ...   
1219                150.2               16.35             83.82   10   18   
1220                103.6               15.15             78.73    6   10   
1221                 73.2               16.52             75.62    4    6   
1222                 96.2               19.02             75.58    5    4   
1223                 52.2               20.99             70.63    4    5   

      t-3  densidade_demografica  
0       5               10646.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 46.8912628	total: 4.04ms	remaining: 4.04s
1:	learn: 45.5363326	total: 9.29ms	remaining: 4.64s
2:	learn: 44.1835761	total: 12.6ms	remaining: 4.18s
3:	learn: 42.7592978	total: 16.1ms	remaining: 4.02s
4:	learn: 41.4430333	total: 18.9ms	remaining: 3.76s
5:	learn: 40.4010948	total: 21.9ms	remaining: 3.64s
6:	learn: 39.3966656	total: 25.1ms	remaining: 3.57s
7:	learn: 38.4645438	total: 28.1ms	remaining: 3.49s
8:	learn: 37.4340414	total: 31.4ms	remaining: 3.46s
9:	learn: 36.4506730	total: 34.5ms	remaining: 3.41s
10:	learn: 35.6531722	total: 36.8ms	remaining: 3.31s
11:	learn: 34.7249933	total: 39.6ms	remaining: 3.26s
12:	learn: 33.8753963	total: 43.4ms	remaining: 3.29s
13:	learn: 33.2518700	total: 46.8ms	remaining: 3.29s
14:	learn: 32.6922054	total: 49.6ms	remaining: 3.26s
15:	learn: 32.0283399	total: 52.4ms	remaining: 3.22s
16:	learn: 31.4476635	total: 56.3ms	remaining: 3.25s
17:	learn: 30.7156819	total: 59.2ms	remaining: 3.23s
18:	learn: 29.9520576	total: 61.8ms	remaining: 3.19s
19:

197:	learn: 3.8102482	total: 575ms	remaining: 2.33s
198:	learn: 3.7932003	total: 579ms	remaining: 2.33s
199:	learn: 3.7811473	total: 583ms	remaining: 2.33s
200:	learn: 3.7732746	total: 586ms	remaining: 2.33s
201:	learn: 3.7628688	total: 590ms	remaining: 2.33s
202:	learn: 3.7307031	total: 593ms	remaining: 2.33s
203:	learn: 3.7219889	total: 596ms	remaining: 2.33s
204:	learn: 3.7006793	total: 602ms	remaining: 2.33s
205:	learn: 3.6848653	total: 607ms	remaining: 2.34s
206:	learn: 3.6629348	total: 610ms	remaining: 2.34s
207:	learn: 3.6455154	total: 614ms	remaining: 2.34s
208:	learn: 3.6303013	total: 617ms	remaining: 2.33s
209:	learn: 3.6213635	total: 619ms	remaining: 2.33s
210:	learn: 3.5948979	total: 623ms	remaining: 2.33s
211:	learn: 3.5729670	total: 626ms	remaining: 2.33s
212:	learn: 3.5538817	total: 629ms	remaining: 2.32s
213:	learn: 3.5292109	total: 633ms	remaining: 2.32s
214:	learn: 3.5256519	total: 635ms	remaining: 2.32s
215:	learn: 3.5086238	total: 639ms	remaining: 2.32s
216:	learn: 

356:	learn: 1.7819607	total: 1.45s	remaining: 2.61s
357:	learn: 1.7666264	total: 1.46s	remaining: 2.61s
358:	learn: 1.7593335	total: 1.46s	remaining: 2.61s
359:	learn: 1.7554976	total: 1.47s	remaining: 2.6s
360:	learn: 1.7476585	total: 1.47s	remaining: 2.6s
361:	learn: 1.7406615	total: 1.47s	remaining: 2.6s
362:	learn: 1.7285609	total: 1.48s	remaining: 2.59s
363:	learn: 1.7105741	total: 1.48s	remaining: 2.59s
364:	learn: 1.6968191	total: 1.49s	remaining: 2.58s
365:	learn: 1.6905889	total: 1.49s	remaining: 2.58s
366:	learn: 1.6851568	total: 1.49s	remaining: 2.58s
367:	learn: 1.6675529	total: 1.5s	remaining: 2.57s
368:	learn: 1.6593536	total: 1.5s	remaining: 2.56s
369:	learn: 1.6509034	total: 1.5s	remaining: 2.56s
370:	learn: 1.6465744	total: 1.51s	remaining: 2.56s
371:	learn: 1.6428608	total: 1.51s	remaining: 2.55s
372:	learn: 1.6375933	total: 1.51s	remaining: 2.54s
373:	learn: 1.6332743	total: 1.52s	remaining: 2.54s
374:	learn: 1.6263646	total: 1.52s	remaining: 2.53s
375:	learn: 1.6252

...

362:	learn: 1.7478971	total: 1.16s	remaining: 2.03s
363:	learn: 1.7429761	total: 1.16s	remaining: 2.03s
364:	learn: 1.7400011	total: 1.16s	remaining: 2.02s
365:	learn: 1.7383419	total: 1.17s	remaining: 2.02s
366:	learn: 1.7244247	total: 1.17s	remaining: 2.02s
367:	learn: 1.7231273	total: 1.17s	remaining: 2.02s
368:	learn: 1.7128733	total: 1.18s	remaining: 2.01s
369:	learn: 1.7085003	total: 1.18s	remaining: 2.01s
370:	learn: 1.7021276	total: 1.19s	remaining: 2.01s
371:	learn: 1.6962330	total: 1.19s	remaining: 2.01s
372:	learn: 1.6952662	total: 1.19s	remaining: 2s
373:	learn: 1.6848994	total: 1.2s	remaining: 2s
374:	learn: 1.6786771	total: 1.2s	remaining: 2s
375:	learn: 1.6771845	total: 1.2s	remaining: 2s
376:	learn: 1.6751293	total: 1.21s	remaining: 1.99s
377:	learn: 1.6559085	total: 1.21s	remaining: 1.99s
378:	learn: 1.6437321	total: 1.21s	remaining: 1.99s
379:	learn: 1.6330501	total: 1.22s	remaining: 1.98s
380:	learn: 1.6254676	total: 1.22s	remaining: 1.98s
381:	learn: 1.6136532	total

530:	learn: 0.9738537	total: 1.77s	remaining: 1.57s
531:	learn: 0.9715126	total: 1.78s	remaining: 1.56s
532:	learn: 0.9682737	total: 1.78s	remaining: 1.56s
533:	learn: 0.9672827	total: 1.79s	remaining: 1.56s
534:	learn: 0.9640655	total: 1.79s	remaining: 1.55s
535:	learn: 0.9619187	total: 1.79s	remaining: 1.55s
536:	learn: 0.9603370	total: 1.8s	remaining: 1.55s
537:	learn: 0.9594544	total: 1.8s	remaining: 1.55s
538:	learn: 0.9576649	total: 1.81s	remaining: 1.54s
539:	learn: 0.9561525	total: 1.81s	remaining: 1.54s
540:	learn: 0.9542190	total: 1.81s	remaining: 1.54s
541:	learn: 0.9519813	total: 1.82s	remaining: 1.54s
542:	learn: 0.9490891	total: 1.82s	remaining: 1.53s
543:	learn: 0.9465845	total: 1.83s	remaining: 1.53s
544:	learn: 0.9450592	total: 1.83s	remaining: 1.53s
545:	learn: 0.9422297	total: 1.83s	remaining: 1.53s
546:	learn: 0.9400822	total: 1.84s	remaining: 1.52s
547:	learn: 0.9376832	total: 1.84s	remaining: 1.52s
548:	learn: 0.9347660	total: 1.84s	remaining: 1.51s
549:	learn: 0.

696:	learn: 0.6624903	total: 2.39s	remaining: 1.04s
697:	learn: 0.6619484	total: 2.4s	remaining: 1.04s
698:	learn: 0.6607430	total: 2.4s	remaining: 1.03s
699:	learn: 0.6581130	total: 2.4s	remaining: 1.03s
700:	learn: 0.6574175	total: 2.41s	remaining: 1.03s
701:	learn: 0.6557665	total: 2.41s	remaining: 1.02s
702:	learn: 0.6550266	total: 2.42s	remaining: 1.02s
703:	learn: 0.6543455	total: 2.42s	remaining: 1.02s
704:	learn: 0.6528149	total: 2.43s	remaining: 1.02s
705:	learn: 0.6507078	total: 2.43s	remaining: 1.01s
706:	learn: 0.6492461	total: 2.44s	remaining: 1.01s
707:	learn: 0.6472435	total: 2.44s	remaining: 1s
708:	learn: 0.6460874	total: 2.44s	remaining: 1s
709:	learn: 0.6443373	total: 2.44s	remaining: 999ms
710:	learn: 0.6434565	total: 2.45s	remaining: 996ms
711:	learn: 0.6416219	total: 2.45s	remaining: 992ms
712:	learn: 0.6397802	total: 2.46s	remaining: 988ms
713:	learn: 0.6393469	total: 2.46s	remaining: 985ms
714:	learn: 0.6385762	total: 2.46s	remaining: 981ms
715:	learn: 0.6367093

865:	learn: 0.4836772	total: 3.08s	remaining: 477ms
866:	learn: 0.4831888	total: 3.09s	remaining: 473ms
867:	learn: 0.4827337	total: 3.09s	remaining: 470ms
868:	learn: 0.4822406	total: 3.09s	remaining: 466ms
869:	learn: 0.4813969	total: 3.1s	remaining: 463ms
870:	learn: 0.4804932	total: 3.1s	remaining: 460ms
871:	learn: 0.4795712	total: 3.11s	remaining: 456ms
872:	learn: 0.4786117	total: 3.11s	remaining: 452ms
873:	learn: 0.4778118	total: 3.12s	remaining: 449ms
874:	learn: 0.4769350	total: 3.12s	remaining: 446ms
875:	learn: 0.4754551	total: 3.13s	remaining: 442ms
876:	learn: 0.4747667	total: 3.13s	remaining: 439ms
877:	learn: 0.4741112	total: 3.13s	remaining: 435ms
878:	learn: 0.4731088	total: 3.14s	remaining: 432ms
879:	learn: 0.4722561	total: 3.14s	remaining: 428ms
880:	learn: 0.4716216	total: 3.15s	remaining: 425ms
881:	learn: 0.4710827	total: 3.15s	remaining: 421ms
882:	learn: 0.4700938	total: 3.15s	remaining: 418ms
883:	learn: 0.4693520	total: 3.16s	remaining: 414ms
884:	learn: 0.

16                         0  
